<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word embedding

In [ ]:
import re
import bs4 as bs
import numpy as np
import pandas as pd
import multiprocessing
import urllib.request
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from keras.preprocessing.text import text_to_word_sequence
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Para este ejercicio vamos a entrenar con artículos de wikipedia todos relacionados con aviación y ver cómo se relacionan entre sí. 

In [ ]:
corpus_text=''
corpus_sentence=[]

In [ ]:
tres_veinte_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Airbus_A320_family')
fh_html=urllib.request.urlopen('https://en.wikipedia.org/wiki/Airport')
ava_html=urllib.request.urlopen('https://en.wikipedia.org/wiki/Avianca')
flight_html=urllib.request.urlopen('https://en.wikipedia.org/wiki/Flight')
faa_html=urllib.request.urlopen('https://en.wikipedia.org/wiki/Federal_Aviation_Administration')

textos=[tres_veinte_html,fh_html,ava_html,flight_html,faa_html]

for texto in textos:
  raw_html = texto.read()
  article_html = bs.BeautifulSoup(raw_html, 'lxml')
  article_paragraphs = article_html.find_all('p')

  for parr in article_paragraphs:
    corpus_text+=parr.text
  
  corpus_sentence.append(corpus_text)

In [ ]:
corpus_sentence

['\nThe Airbus A320 family is a series of narrow-body airliners developed and produced by Airbus.\nThe A320 was launched in March 1984, first flew on 22 February 1987, and was introduced in April 1988 by Air France.\nThe first member of the family was followed by the longer A321 (first delivered in January 1994), the shorter A319 (April 1996), and the even shorter A318 (July 2003).\nFinal assembly takes place in Toulouse in France; Hamburg in Germany; Tianjin in China since 2009; and in Mobile, Alabama in the United States since April 2016.\nThe twinjet has a six-abreast cross-section and is powered by either CFM56 or IAE V2500 turbofans, except the CFM56/PW6000 powered A318.\nThe family pioneered the use of digital fly-by-wire and side-stick flight controls in airliners.\nVariants offer maximum take-off weights from 68 to 93.5\xa0t (150,000 to 206,000\xa0lb), to cover a 5,740–6,940\xa0km (3,100–3,750\xa0nmi) range.\nThe 31.4\xa0m (103\xa0ft) long A318 typically accommodates 107 to 132

In [ ]:
#Revisemos qué obtuvimos
corpus_text

'\nThe Airbus A320 family is a series of narrow-body airliners developed and produced by Airbus.\nThe A320 was launched in March 1984, first flew on 22 February 1987, and was introduced in April 1988 by Air France.\nThe first member of the family was followed by the longer A321 (first delivered in January 1994), the shorter A319 (April 1996), and the even shorter A318 (July 2003).\nFinal assembly takes place in Toulouse in France; Hamburg in Germany; Tianjin in China since 2009; and in Mobile, Alabama in the United States since April 2016.\nThe twinjet has a six-abreast cross-section and is powered by either CFM56 or IAE V2500 turbofans, except the CFM56/PW6000 powered A318.\nThe family pioneered the use of digital fly-by-wire and side-stick flight controls in airliners.\nVariants offer maximum take-off weights from 68 to 93.5\xa0t (150,000 to 206,000\xa0lb), to cover a 5,740–6,940\xa0km (3,100–3,750\xa0nmi) range.\nThe 31.4\xa0m (103\xa0ft) long A318 typically accommodates 107 to 132 

Hacemos el preprocesamiento de los datos:

In [ ]:
article_text = corpus_text.lower()
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [ ]:
article_text

'\nthe airbus a320 family is a series of narrow-body airliners developed and produced by airbus.\nthe a320 was launched in march 1984, first flew on 22 february 1987, and was introduced in april 1988 by air france.\nthe first member of the family was followed by the longer a321 (first delivered in january 1994), the shorter a319 (april 1996), and the even shorter a318 (july 2003).\nfinal assembly takes place in toulouse in france; hamburg in germany; tianjin in china since 2009; and in mobile, alabama in the united states since april 2016.\nthe twinjet has a six-abreast cross-section and is powered by either cfm56 or iae v2500 turbofans, except the cfm56/pw6000 powered a318.\nthe family pioneered the use of digital fly-by-wire and side-stick flight controls in airliners.\nvariants offer maximum take-off weights from 68 to 93.5\xa0t (150,000 to 206,000\xa0lb), to cover a 5,740–6,940\xa0km (3,100–3,750\xa0nmi) range.\nthe 31.4\xa0m (103\xa0ft) long a318 typically accommodates 107 to 132 

In [ ]:
text

' the airbus a320 family is a series of narrow-body airliners developed and produced by airbus. the a320 was launched in march 1984, first flew on 22 february 1987, and was introduced in april 1988 by air france. the first member of the family was followed by the longer a321 (first delivered in january 1994), the shorter a319 (april 1996), and the even shorter a318 (july 2003). final assembly takes place in toulouse in france; hamburg in germany; tianjin in china since 2009; and in mobile, alabama in the united states since april 2016. the twinjet has a six-abreast cross-section and is powered by either cfm56 or iae v2500 turbofans, except the cfm56/pw6000 powered a318. the family pioneered the use of digital fly-by-wire and side-stick flight controls in airliners. variants offer maximum take-off weights from 68 to 93.5 t (150,000 to 206,000 lb), to cover a 5,740–6,940 km (3,100–3,750 nmi) range. the 31.4 m (103 ft) long a318 typically accommodates 107 to 132 passengers. the 124-156 se

Tokenizamos el texto

In [ ]:
corpus_token = nltk.sent_tokenize(text)
words_token = nltk.word_tokenize(text)

In [ ]:
sentence_tokens=[]

#for txto in text():
#  sentence_tokens.append(text_to_word_sequence(txto)

k=text_to_word_sequence(text)

In [ ]:
k

['the',
 'airbus',
 'a320',
 'family',
 'is',
 'a',
 'series',
 'of',
 'narrow',
 'body',
 'airliners',
 'developed',
 'and',
 'produced',
 'by',
 'airbus',
 'the',
 'a320',
 'was',
 'launched',
 'in',
 'march',
 '1984',
 'first',
 'flew',
 'on',
 '22',
 'february',
 '1987',
 'and',
 'was',
 'introduced',
 'in',
 'april',
 '1988',
 'by',
 'air',
 'france',
 'the',
 'first',
 'member',
 'of',
 'the',
 'family',
 'was',
 'followed',
 'by',
 'the',
 'longer',
 'a321',
 'first',
 'delivered',
 'in',
 'january',
 '1994',
 'the',
 'shorter',
 'a319',
 'april',
 '1996',
 'and',
 'the',
 'even',
 'shorter',
 'a318',
 'july',
 '2003',
 'final',
 'assembly',
 'takes',
 'place',
 'in',
 'toulouse',
 'in',
 'france',
 'hamburg',
 'in',
 'germany',
 'tianjin',
 'in',
 'china',
 'since',
 '2009',
 'and',
 'in',
 'mobile',
 'alabama',
 'in',
 'the',
 'united',
 'states',
 'since',
 'april',
 '2016',
 'the',
 'twinjet',
 'has',
 'a',
 'six',
 'abreast',
 'cross',
 'section',
 'and',
 'is',
 'powered',

In [ ]:
corpus_token[:10]

[' the airbus a320 family is a series of narrow-body airliners developed and produced by airbus.',
 'the a320 was launched in march 1984, first flew on 22 february 1987, and was introduced in april 1988 by air france.',
 'the first member of the family was followed by the longer a321 (first delivered in january 1994), the shorter a319 (april 1996), and the even shorter a318 (july 2003).',
 'final assembly takes place in toulouse in france; hamburg in germany; tianjin in china since 2009; and in mobile, alabama in the united states since april 2016. the twinjet has a six-abreast cross-section and is powered by either cfm56 or iae v2500 turbofans, except the cfm56/pw6000 powered a318.',
 'the family pioneered the use of digital fly-by-wire and side-stick flight controls in airliners.',
 'variants offer maximum take-off weights from 68 to 93.5 t (150,000 to 206,000 lb), to cover a 5,740–6,940 km (3,100–3,750 nmi) range.',
 'the 31.4 m (103 ft) long a318 typically accommodates 107 to 132 p

In [ ]:
words_token[:20]

['the',
 'airbus',
 'a320',
 'family',
 'is',
 'a',
 'series',
 'of',
 'narrow-body',
 'airliners',
 'developed',
 'and',
 'produced',
 'by',
 'airbus',
 '.',
 'the',
 'a320',
 'was',
 'launched']

In [ ]:
lemmatizer = WordNetLemmatizer()
def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

Creamos los vectores

In [ ]:
#Hacemos esto de forma que podamos ver el loss de cada época
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)   

In [ ]:
w2v_model.build_vocab([k])

Entrenamos el modelos

In [ ]:
w2v_model.train([k],
                 total_examples=w2v_model.corpus_count,
                 epochs=1000,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 219906.640625
Loss after epoch 1: 118689.578125
Loss after epoch 2: 118066.5625
Loss after epoch 3: 118188.46875
Loss after epoch 4: 118262.8125
Loss after epoch 5: 119974.75
Loss after epoch 6: 119975.4375
Loss after epoch 7: 118625.125
Loss after epoch 8: 106067.625
Loss after epoch 9: 105873.25
Loss after epoch 10: 103683.5
Loss after epoch 11: 103160.875
Loss after epoch 12: 101681.625
Loss after epoch 13: 100398.875
Loss after epoch 14: 99913.875
Loss after epoch 15: 99083.125
Loss after epoch 16: 98046.75
Loss after epoch 17: 96393.375
Loss after epoch 18: 88039.75
Loss after epoch 19: 83973.0
Loss after epoch 20: 83148.5
Loss after epoch 21: 81942.5
Loss after epoch 22: 81142.75
Loss after epoch 23: 80652.25
Loss after epoch 24: 79999.75
Loss after epoch 25: 79203.75
Loss after epoch 26: 78234.5
Loss after epoch 27: 77189.0
Loss after epoch 28: 76606.0
Loss after epoch 29: 76227.75
Loss after epoch 30: 75058.0
Loss after epoch 31: 74185.75
Loss after epoch 32

(10000000, 23225000)

In [ ]:
#Buscamos la palabra más cercana a Avianca
w2v_model.wv.most_similar(positive=["avianca"], topn=10)

[('cab', 0.35681650042533875),
 ('colombia', 0.35347074270248413),
 ("avianca's", 0.2926785349845886),
 ('airline', 0.2911369204521179),
 ('taca', 0.29065507650375366),
 ('group', 0.2874971330165863),
 ('york', 0.2864956259727478),
 ('announced', 0.28468242287635803),
 ('december', 0.2807961404323578),
 ('bankruptcy', 0.2800130546092987)]

In [ ]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
